# Retrieve all the relevant data from the table athletic-data in the database.
# Pre-requisite :
  1. Activity has to be of more than 2hrs.
  2. Watts and HeartRate column should not be null.

In [1]:
from src.repositories.PowerAndHRRepository import *

athletic_data_db = get_athletic_data()

You are successfully connected to your Database!


/Users/lalitpoddar/Desktop/Uni/MS Project Thesis/hr-power-analysis/src/repositories/PowerAndHRRepository.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(configs.get("athletic-record-query").data, hr_power_db_conn)


# DATA PROFILING : Removing irrelevant columns not required for analysis.

In [2]:
cols_to_ignore = configs.get("ignore-columns").data
athletic_data = athletic_data_db.drop(cols_to_ignore.split(","), axis=1)

# DATA CLEANSING : Filling missing data with immediate pre- and post-values' mean

In [3]:
def update_NA_values(lst):
    for i in range(len(lst)):
        if lst[i] is None:
            j = i - 1
            while j >= 0 and lst[j] is None:
                j -= 1
            previous_value = lst[j] if j >= 0 else None
            j = i + 1
            while j < len(lst) and lst[j] is None:
                j += 1
            next_value = lst[j] if j < len(lst) else None
            if previous_value is not None and next_value is not None:
                lst[i] = (previous_value + next_value) / 2
            elif previous_value is not None:
                lst[i] = previous_value
            elif next_value is not None:
                lst[i] = next_value
    return lst


athletic_data['heartrate'] = athletic_data['heartrate'].apply(update_NA_values)
athletic_data['watts'] = athletic_data['watts'].apply(update_NA_values)


# Remove rows with more than half of values as None in heartrate or watts lists
athletic_data = athletic_data[(athletic_data['heartrate'].apply(lambda x: x.count(None) <= len(x) / 2)) &
            (athletic_data['watts'].apply(lambda x: x.count(None) <= len(x) / 2))]

# Filter the DataFrame based on the activity_id
filtered_data = athletic_data[athletic_data['activity_id'] == 9057354882]

# Get the unique values in the 'watts' list
unique_values = set(filtered_data['watts'].sum())

print(unique_values)